In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from networks.vit_seg_modeling import VisionTransformer as ViT
from networks.vit_seg_modeling import CONFIGS as CONFIGS_ViT_seg

ModuleNotFoundError: No module named 'ml_collections'

In [22]:
config_vit = CONFIGS_ViT_seg['R50-ViT-B_16']
config_vit.n_classes = 2
config_vit.n_skip = 3
config_vit.patches.size = (16,16)
config_vit.patches.grid = (int(16), int(16))
transnuss = ViT(config_vit, 256, 2)
class Model_(nn.Module):
    def __init__(self, transnuss: nn.Module, color_unet: nn.Module, str_unet: nn.Module):
        self.transnuss = transnuss
        self.color_unet = color_unet
        self.str_unet = str_unet
        # self.patch_model.load_state_dict(torch.load(t_checkpoint))
        # self.color_unet.load_state_dict(torch.load(c_checkpoint))
        # self.str_unet.load_state_dict(torch.load(s_checkpoint))
        self.conv_layer = nn.Sequential(nn.Conv2d(1, 8, 3, 1, 1),
                                        nn.ELU(),
                                        nn.BatchNorm2d(8),
                                        nn.Conv2d(8, 1, 3, 1, 1),
                                        nn.Sigmoid())

    def forward(self, x):
        tr_out = self.transnuss(x)
        unet_out = self.color_unet(x)
        st_out = self.str_unet(x)
        return self.conv_layer(torch.stack((patch_out, unet_out), dim=1))